In [1]:
# Script to scrape card usage and win-rate statistics from RoyaleAPI and save them to a CSV file
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Base URL for the card popularity table (7-day window, English language, table display mode)
base_url = "https://royaleapi.com/cards/popular?lang=en&time=7d&mode=table"
# Mapping of bracket labels to specific RoyaleAPI category endpoints
urls = {
    "Top 200":   base_url + "&cat=TopRanked200&sort=usage",
    "Top 1000":  base_url + "&cat=TopRanked1000&sort=usage",
    "All Ranked": base_url + "&cat=Ranked&sort=usage",
    "Ladder": base_url +  "&cat=Ladder&sort=usage"
}

# HTTP request headers; sets a User-Agent so the request looks like it is coming from a browser
headers = {
    "User-Agent": "Mozilla/5.0 (compatible; card-scraper/1.0; +https://example.com)"
}

# Container to collect all parsed records across brackets
all_records = []

# Iterate over each bracket endpoint and download its HTML page
for bracket, url in urls.items():
    # Send the HTTP GET request with custom headers and a timeout
    resp = requests.get(url, headers=headers, timeout=15)
    # Raise an exception if the HTTP response returned an error status code
    resp.raise_for_status()

    # Parse the HTML content of the response with BeautifulSoup
    soup = BeautifulSoup(resp.text, "html.parser")
    # Locate the first table element that contains the card statistics
    table = soup.find("table")

    # Get all table rows from the statistics table
    rows = table.find_all("tr")


    # Skip the header row (index 0) and iterate over the remaining data rows
    for row in rows[1:]:
        # Extract text from each cell in the current row
        cells = [c.get_text(strip=True) for c in row.find_all(["td", "th"])]

        # Skip rows that do not have the expected number of columns
        if len(cells) < 7:
            continue

        # Extract the card name, rating, usage percentage, and win percentage from the row
        card_name = cells[1]
        rating = cells[2]
        usage_raw = cells[3].replace("%", "")
        win_raw = cells[5].replace("%", "")

        # Helper function to convert a string to float, returning None when conversion fails
        def to_float_safe(s):
            try:
                return float(s)
            except ValueError:
                return None

        # Convert the cleaned usage and win percentage strings into numeric values
        usage_pct = to_float_safe(usage_raw)
        win_pct = to_float_safe(win_raw)

        # Append the normalized record for this card and bracket to the main list
        all_records.append({
            "CardName": card_name,
            "Bracket": bracket,
            "UsagePct": usage_pct,
            "WinPct": win_pct,
            "Rating": rating,
        })

# Create a pandas DataFrame from all collected records
df = pd.DataFrame(all_records)
# Save the aggregated card statistics to a CSV file
df.to_csv("card_stats_royalapi.csv", index=False)
